# Import Libraries

In [ ]:
import zipfile
import os
import random
import base64
from openai import OpenAI
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt
from google.colab import userdata
import os
import re

In [ ]:
from google.colab import userdata
import os

os.environ['OPENAI_API_KEY'] = '***'

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# 1. Unzip dataset
zip_path = "/content/drive/MyDrive/Data2.zip"
unzip_dir = "/content/data2"

In [ ]:
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(unzip_dir)

In [ ]:
import base64
from openai import OpenAI

client = OpenAI()

# Function to encode the image
def encode_image(image_path):
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode("utf-8")

In [ ]:
# 1) Gather all image paths and their true labels
all_images = []
true_labels = []
dir_paths = {
    "YES": "/content/data2/Data/Data/Guardrail/Yes",
    "NO":  "/content/data2/Data/Data/Guardrail/No"
}


for label, folder in dir_paths.items():
    for fname in os.listdir(folder):
        if fname.lower().endswith((".jpg", ".jpeg", ".png")):
            all_images.append(os.path.join(folder, fname))
            true_labels.append(label)

In [ ]:
def make_request_few_shot(base64_image, positive_shots: list, negative_shots: list):
    negative_images = [
        {
            "type": "image_url",
            "image_url": {
                "url": f"data:image/jpeg;base64,{tmp_img}",
                "detail": "low"
            },
        } for tmp_img in negative_shots
    ]

    positive_images = [
        {
            "type": "image_url",
            "image_url": {
                "url": f"data:image/jpeg;base64,{tmp_img}",
                "detail": "low"
            },
        } for tmp_img in positive_shots
    ]

    content = (
        [{"type": "text", "text": "Examples of images labeled NO (no guardrails or New Jersey barriers present):"}] +
        negative_images +
        [{"type": "text", "text": "Examples of images labeled YES (guardrails or New Jersey barriers are present):"}] +
        positive_images +
        [{"type": "text", "text": "First tell me what you see. Answer YES OR NO strictly after telling me your reasoning."}] +
        [{
            "type": "image_url",
            "image_url": {
                "url": f"data:image/jpeg;base64,{base64_image}",
                "detail": "low"
            },
        }]
    )

    completion = client.chat.completions.create(
        model="gpt-4.1",
        messages=[
            {
                "role": "user",
                "content": content,
            },
        ],
    )

    response_text = completion.choices[0].message.content
    print(response_text)  # This is optional for logging
    return response_text  # ✅ This is the actual fix!

In [ ]:
def load_and_encode_images(folder_path):
    encoded_images = []
    for filename in os.listdir(folder_path):
        if filename.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.gif')):
            image_path = os.path.join(folder_path, filename)
            encoded_images.append(encode_image(image_path))
    return encoded_images

# Update with your actual folder paths
yes_folder = '/content/drive/MyDrive/supported images/YES'
no_folder = '/content/drive/MyDrive/supported images/NO'


encoded_yes_images = load_and_encode_images(yes_folder)
encoded_no_images = load_and_encode_images(no_folder)

In [ ]:
# def extract_final_answer(response):
#     if not isinstance(response, str):
#         return "UNKNOWN"

#     # Prioritize structured answer like: **Answer: YES**
#     match = re.search(r"\banswer\s*[:\-]?\s*(YES|NO)\b", response, re.IGNORECASE)
#     if match:
#         return match.group(1).upper()

#     # Fallback: any standalone YES or NO
#     fallback = re.search(r"\b(YES|NO)\b", response.upper())
#     return fallback.group(1) if fallback else "UNKNOWN"

In [ ]:
# import re

# def extract_final_answer(response):
#     if not isinstance(response, str):
#         return "UNKNOWN"

#     response = response.strip()

#     # 1. Strong match: clearly labeled final answer
#     structured_match = re.search(
#         r"\b(?:final\s*)?(?:answer|response)\s*[:\-]?\s*(YES|NO)\b", response, re.IGNORECASE
#     )
#     if structured_match:
#         return structured_match.group(1).upper()

#     # 2. Flexible phrasing: common language leading to YES/NO
#     phrased_match = re.search(
#         r"\b(?:the\s+)?(?:answer|response)\s+(?:is|would\s+be|should\s+be)\s+(YES|NO)\b",
#         response, re.IGNORECASE
#     )
#     if phrased_match:
#         return phrased_match.group(1).upper()

#     # 3. Standalone YES/NO lines
#     for line in response.splitlines():
#         cleaned = line.strip().upper()
#         if cleaned in {"YES", "NO"}:
#             return cleaned

#     # 4. Final fallback: any YES/NO appearing alone near the end
#     fallback = re.search(r"\b(YES|NO)\b", response[-50:].upper())  # Focus only on ending part
#     if fallback:
#         return fallback.group(1)

#     return "UNKNOWN"

In [ ]:
# import re

# def extract_final_answer(response):
#     if not isinstance(response, str):
#         return "NO"  # fallback to majority logic — pick "NO" if nothing makes sense at all

#     response = response.strip()

#     # 1. Strong match: "final answer: yes", etc.
#     structured_match = re.search(
#         r"\b(?:final\s*)?(?:answer|response)\s*[:\-]?\s*(YES|NO)\b", response, re.IGNORECASE
#     )
#     if structured_match:
#         return structured_match.group(1).upper()

#     # 2. Flexible phrasing
#     phrased_match = re.search(
#         r"\b(?:the\s+)?(?:answer|response)\s+(?:is|would\s+be|should\s+be)\s+(YES|NO)\b",
#         response, re.IGNORECASE
#     )
#     if phrased_match:
#         return phrased_match.group(1).upper()

#     # 3. Standalone YES/NO lines
#     for line in response.splitlines():
#         cleaned = line.strip().upper()
#         if cleaned in {"YES", "NO"}:
#             return cleaned

#     # 4. Fallback: use frequency count in whole response
#     yes_count = len(re.findall(r"\bYES\b", response.upper()))
#     no_count = len(re.findall(r"\bNO\b", response.upper()))

#     if yes_count > no_count:
#         return "YES"
#     elif no_count > yes_count:
#         return "NO"
#     else:
#         return "NO"  # consistent tiebreaker if counts are equal

In [ ]:
import re

def extract_final_answer(response: str) -> str:
    """
    Extracts a final YES/NO from a reasoning+answer text.
    Returns "YES", "NO", or "NO" as a default fallback.
    """
    if not isinstance(response, str) or not response.strip():
        return "NO"

    # 0. Normalize whitespace, unify line endings
    text = response.replace("\r\n", "\n").strip()

    # 1. Quick clean-up: correct common typos and strip Markdown emphasis
    #    - fix 'asnwer' → 'answer'
    #    - remove surrounding * or ** around YES/NO
    text = re.sub(r"(?i)\basnwer\b", "answer", text)
    text = re.sub(r"[*_]*(YES|NO)[*_]*", r"\1", text, flags=re.IGNORECASE)

    # 2. Scan from bottom to top for explicit final-answer labels
    lines = text.split("\n")
    for line in reversed(lines):
        # a) Structured: final answer: YES / final-answer – NO / final answer – YES
        m = re.search(
            r"\bfinal[-\s]*answer\s*[:\-–]?\s*(YES|NO)\b",
            line,
            re.IGNORECASE,
        )
        if m:
            return m.group(1).upper()

        # b) Flexible: answer: YES / answer – NO
        m = re.search(
            r"\banswer\s*[:\-–]?\s*(YES|NO)\b",
            line,
            re.IGNORECASE,
        )
        if m:
            return m.group(1).upper()

        # c) Phrased: the answer is YES / answer would be NO
        m = re.search(
            r"\b(?:the\s+)?answer\s+(?:is|would\s+be|should\s+be)\s+(YES|NO)\b",
            line,
            re.IGNORECASE,
        )
        if m:
            return m.group(1).upper()

        # d) Standalone
        if line.strip().upper() in {"YES", "NO"}:
            return line.strip().upper()

    # 3. Fallback to frequency count
    upper = text.upper()
    yes_count = len(re.findall(r"\bYES\b", upper))
    no_count  = len(re.findall(r"\bNO\b", upper))
    if yes_count > no_count:
        return "YES"
    # tie or more NOs
    return "NO"

In [ ]:
from concurrent.futures import ThreadPoolExecutor
from itertools import repeat
import pandas as pd

batch_size = 40
pred_labels = []
valid_true_labels = []
results = []

def process_batch(batch_imgs, batch_trues, encoded_yes_images, encoded_no_images):
    base64_batch = [encode_image(p) for p in batch_imgs]

    with ThreadPoolExecutor(max_workers=batch_size) as executor:
        responses = executor.map(
            make_request_few_shot,
            base64_batch,
            repeat(encoded_yes_images),
            repeat(encoded_no_images),
        )

    for img_path, true_label, response in zip(batch_imgs, batch_trues, responses):
        if response is None:
            print(f"❌ No response from model: {img_path}")
            continue

        print("📝 Raw Response:", response)
        pred = extract_final_answer(response)

        if pred in ["YES", "NO"]:
            pred_labels.append(pred)
            valid_true_labels.append(true_label)
            print(f"✅ Prediction: {pred} | True: {true_label}")
        else:
            pred = "UNKNOWN"
            print(f"⚠️ Could not extract YES/NO: {img_path}")

        results.append({
            "image_id": img_path,
            "true_label": true_label,
            "prediction": pred,
            "reasoning": response
        })


# Execute batching logic
for start in range(0, len(all_images), batch_size):
    batch_imgs = all_images[start : start + batch_size]
    batch_trues = true_labels[start : start + batch_size]

    process_batch(batch_imgs, batch_trues, encoded_yes_images, encoded_no_images)

In [ ]:
# 7. Confusion matrix (only on valid results)
cm = confusion_matrix(valid_true_labels, pred_labels, labels=["YES", "NO"])
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=["YES", "NO"])
disp.plot(cmap=plt.cm.Blues)
plt.title("Confusion Matrix")
plt.show()

In [ ]:
# Classification report
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, classification_report
report = classification_report(valid_true_labels, pred_labels, labels=["YES", "NO"])
print("Classification Report:")
print(report)